In [30]:
import random
import time
from faker import Faker
from typing import List, Dict, Any

class BasketballAPI:
    def __init__(self, players_per_season=390):
        print("Initializing Basketball API...")
        # Initialize basic parameters
        self.players_per_season = players_per_season
        self.elite_percentage = 0.05
        self.movement_probability = 0.15
        self.faker = Faker()
        Faker.seed(12345)  # For reproducibility
        
        # Define seasons and teams
        self.seasons = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
        self.teams = ['SkyBirds', 'VBADragons', 'SparkiFY', 'KeyBoardWarriors', 'UnitedWeCode', 'HtmlERS', 'LutammS', 'BIsupremE']
        # Generate players and their attributes
        self._initialize_players()
        self._assign_teams_by_season()
        print("Initialization complete!")

    def _initialize_players(self):
        """Generate exactly 390 players and assign their attributes"""
        print("Generating players...")
        self.players = []
        unique_names = set()
        
        while len(unique_names) < self.players_per_season:
            name = f"{self.faker.first_name()} {self.faker.last_name()}"
            if name not in unique_names:
                unique_names.add(name)
                self.players.append(name)
        
        # Calculate number of elite players
        num_elite = int(self.players_per_season * self.elite_percentage)
        
        print(f"Assigning player archetypes ({num_elite} players per elite category)...")
        # Randomly select elite players
        all_players = set(self.players)
        self.elite_scorers = set(random.sample(list(all_players), num_elite))
        remaining_players = all_players - self.elite_scorers
        
        self.elite_defenders = set(random.sample(list(remaining_players), num_elite))
        remaining_players = remaining_players - self.elite_defenders
        
        self.elite_playmakers = set(random.sample(list(remaining_players), num_elite))
        self.regular_players = remaining_players - self.elite_playmakers
        
        # Create player dictionary with attributes
        self.player_data = {}
        for idx, player in enumerate(self.players):
            archetype = 'regular'
            if player in self.elite_scorers:
                archetype = 'elite_scorer'
            elif player in self.elite_defenders:
                archetype = 'elite_defender'
            elif player in self.elite_playmakers:
                archetype = 'elite_playmaker'
            
            self.player_data[player] = {
                'player_id': idx,
                'archetype': archetype,
                'movement_prone': random.random() < self.movement_probability
            }

    def _assign_teams_by_season(self):
        """Assign teams to players for each season"""
        print("Assigning teams for each season...")
        self.player_teams = {player: {} for player in self.players}
        
        # Initial team assignment
        for player in self.players:
            self.player_teams[player][self.seasons[0]] = random.choice(self.teams)
        
        # Assign teams for subsequent seasons
        for i in range(1, len(self.seasons)):
            current_season = self.seasons[i]
            previous_season = self.seasons[i-1]
            
            for player in self.players:
                previous_team = self.player_teams[player][previous_season]
                
                if self.player_data[player]['movement_prone'] and random.random() < 0.3:
                    # 30% chance to change teams for movement-prone players
                    new_team = random.choice([t for t in self.teams if t != previous_team])
                    self.player_teams[player][current_season] = new_team
                else:
                    # Stay with the same team
                    self.player_teams[player][current_season] = previous_team

    def _generate_stats(self, player: str, season: str) -> Dict[str, Any]:
        """Generate statistics based on player archetype"""
        archetype = self.player_data[player]['archetype']
        base_stats = {
            'games_played': random.randint(20, 82),
            'minutes': round(random.uniform(12, 38), 1)
        }
        
        if archetype == 'elite_scorer':
            stats = {
                'points': round(random.uniform(20, 35), 1),
                'rebounds': round(random.uniform(1, 8), 1),
                'assists': round(random.uniform(2, 6), 1),
                'steals': round(random.uniform(0, 2), 1),
                'blocks': round(random.uniform(0, 1), 1),
                'fg_pct': round(random.uniform(0.450, 0.650), 3),
                'fg3_pct': round(random.uniform(0.350, 0.450), 3),
                'ft_pct': round(random.uniform(0.800, 0.950), 3)
            }
        elif archetype == 'elite_defender':
            stats = {
                'points': round(random.uniform(8, 15), 1),
                'rebounds': round(random.uniform(5, 12), 1),
                'assists': round(random.uniform(1, 4), 1),
                'steals': round(random.uniform(1.5, 3), 1),
                'blocks': round(random.uniform(1, 3), 1),
                'fg_pct': round(random.uniform(0.350, 0.550), 3),
                'fg3_pct': round(random.uniform(0.250, 0.380), 3),
                'ft_pct': round(random.uniform(0.650, 0.850), 3)
            }
        elif archetype == 'elite_playmaker':
            stats = {
                'points': round(random.uniform(15, 25), 1),
                'rebounds': round(random.uniform(3, 7), 1),
                'assists': round(random.uniform(8, 12), 1),
                'steals': round(random.uniform(1, 2.5), 1),
                'blocks': round(random.uniform(0, 1), 1),
                'fg_pct': round(random.uniform(0.400, 0.550), 3),
                'fg3_pct': round(random.uniform(0.350, 0.420), 3),
                'ft_pct': round(random.uniform(0.800, 0.900), 3)
            }
        else:  # regular players
            tier = random.choices(['rotation', 'bench', 'deep_bench'], 
                                weights=[0.5, 0.3, 0.2])[0]
            
            if tier == 'rotation':
                stats = {
                    'points': round(random.uniform(8, 18), 1),
                    'rebounds': round(random.uniform(3, 8), 1),
                    'assists': round(random.uniform(2, 5), 1),
                    'steals': round(random.uniform(0.5, 1.8), 1),
                    'blocks': round(random.uniform(0.3, 1.2), 1),
                    'fg_pct': round(random.uniform(0.400, 0.520), 3),
                    'fg3_pct': round(random.uniform(0.300, 0.380), 3),
                    'ft_pct': round(random.uniform(0.700, 0.850), 3)
                }
            elif tier == 'bench':
                stats = {
                    'points': round(random.uniform(5, 12), 1),
                    'rebounds': round(random.uniform(2, 5), 1),
                    'assists': round(random.uniform(1, 3), 1),
                    'steals': round(random.uniform(0.3, 1.2), 1),
                    'blocks': round(random.uniform(0.2, 0.8), 1),
                    'fg_pct': round(random.uniform(0.380, 0.480), 3),
                    'fg3_pct': round(random.uniform(0.280, 0.360), 3),
                    'ft_pct': round(random.uniform(0.650, 0.800), 3)
                }
            else:  # deep_bench
                stats = {
                    'points': round(random.uniform(2, 8), 1),
                    'rebounds': round(random.uniform(1, 4), 1),
                    'assists': round(random.uniform(0.5, 2), 1),
                    'steals': round(random.uniform(0.2, 0.8), 1),
                    'blocks': round(random.uniform(0.1, 0.5), 1),
                    'fg_pct': round(random.uniform(0.350, 0.450), 3),
                    'fg3_pct': round(random.uniform(0.250, 0.330), 3),
                    'ft_pct': round(random.uniform(0.600, 0.750), 3)
                }
        
        return {**base_stats, **stats}

    def get_players(self) -> List[Dict[str, Any]]:
        """Method to get player data"""
        print("\nLoading player data...")
        time.sleep(5)  # Simulate 5-second delay for player data
        
        players = [
            {
                'player_id': data['player_id'],
                'player_name': player,
                'archetype': data['archetype']
            }
            for player, data in self.player_data.items()
        ]
        
        print(f"Player data loaded successfully! ({len(players)} players)")
        return players

    def get_stats(self) -> List[Dict[str, Any]]:
        """Method to get player statistics"""
        stats_data = []
        
        print("\nBeginning statistics data load...")
        for season in self.seasons:
            print(f"\nLoading data for season {season}...")
            time.sleep(30)  # Simulate 30-second delay for each season
            
            season_stats = []
            for player in self.players:
                stats = self._generate_stats(player, season)
                season_stats.append({
                    'player_id': self.player_data[player]['player_id'],
                    'season': season,
                    'team': self.player_teams[player][season],
                    **stats
                })
            
            stats_data.extend(season_stats)
            print(f"Loaded {len(season_stats)} player records for season {season}")
        
        print(f"\nAll statistics loaded successfully! ({len(stats_data)} total records)")
        return stats_data


if __name__ == "__main__":
    # Test the API with timing information
    start_time = time.time()
    
    print("Starting data generation process...")
    api = BasketballAPI()
    
    print("\nFetching player data...")
    players = api.get_players()
    
    print("\nFetching statistics...")
    stats = api.get_stats()
    
    end_time = time.time()
    total_time = round(end_time - start_time)
    minutes = total_time // 60
    seconds = total_time % 60
    
    print(f"\nProcess completed in {minutes} minutes and {seconds} seconds")

Starting data generation process...
Initializing Basketball API...
Generating players...
Assigning player archetypes (19 players per elite category)...
Assigning teams for each season...
Initialization complete!

Fetching player data...

Loading player data...
Player data loaded successfully! (390 players)

Fetching statistics...

Beginning statistics data load...

Loading data for season 2019-20...
Loaded 390 player records for season 2019-20

Loading data for season 2020-21...
Loaded 390 player records for season 2020-21

Loading data for season 2021-22...
Loaded 390 player records for season 2021-22

Loading data for season 2022-23...
Loaded 390 player records for season 2022-23

Loading data for season 2023-24...
Loaded 390 player records for season 2023-24

All statistics loaded successfully! (1950 total records)

Process completed in 2 minutes and 35 seconds


In [31]:
from BasketballAPI import BasketballAPI

In [34]:
BasketballAPI?

Init signature: BasketballAPI(players_per_season=390)
Docstring:      <no docstring>
File:           ~/Desktop/UdacityFinalProject/BasketballAPI.py
Type:           type
Subclasses:     

In [24]:
api = BasketballAPI()
players = api.get_stats()
type(players)

Initializing Basketball API...
Generating players...
Assigning player archetypes (19 players per elite category)...
Assigning teams for each season...
Initialization complete!

Beginning statistics data load...

Loading data for season 2019-20...
Loaded 390 player records for season 2019-20

Loading data for season 2020-21...
Loaded 390 player records for season 2020-21

Loading data for season 2021-22...
Loaded 390 player records for season 2021-22

Loading data for season 2022-23...
Loaded 390 player records for season 2022-23

Loading data for season 2023-24...
Loaded 390 player records for season 2023-24

All statistics loaded successfully! (1950 total records)


list

In [26]:
stats = api.get_stats()


Beginning statistics data load...

Loading data for season 2019-20...
Loaded 390 player records for season 2019-20

Loading data for season 2020-21...
Loaded 390 player records for season 2020-21

Loading data for season 2021-22...
Loaded 390 player records for season 2021-22

Loading data for season 2022-23...
Loaded 390 player records for season 2022-23

Loading data for season 2023-24...
Loaded 390 player records for season 2023-24

All statistics loaded successfully! (1950 total records)


In [28]:
stats[0:2]

[{'player_id': 0,
  'season': '2019-20',
  'team': 'Polars',
  'games_played': 77,
  'minutes': 27.1,
  'points': 8.3,
  'rebounds': 5.7,
  'assists': 4.8,
  'steals': 0.9,
  'blocks': 0.5,
  'fg_pct': 0.451,
  'fg3_pct': 0.335,
  'ft_pct': 0.789},
 {'player_id': 1,
  'season': '2019-20',
  'team': 'LobS',
  'games_played': 45,
  'minutes': 13.3,
  'points': 9.0,
  'rebounds': 4.1,
  'assists': 3.1,
  'steals': 1.6,
  'blocks': 0.9,
  'fg_pct': 0.434,
  'fg3_pct': 0.364,
  'ft_pct': 0.731}]

In [18]:
import random
import time
from faker import Faker
from typing import List, Dict, Any

class BasketballAPI:
    def __init__(self, players_per_season=390):
        print("Initializing Basketball API...")
        # Initialize basic parameters
        self.players_per_season = players_per_season
        self.elite_percentage = 0.05
        self.movement_probability = 0.15
        self.faker = Faker()
        Faker.seed(12345)  # For reproducibility
        
        # Define seasons
        self.seasons = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
        
        # Define teams with detailed information
        self.teams = {
            'SKY': {
                'id': 1,
                'name': 'Chicago Skyline',
                'city': 'Chicago',
                'state': 'Illinois',
                'creation_date': '1997-04-15'
            },
            'LEG': {
                'id': 2,
                'name': 'Boston Legacy',
                'city': 'Boston',
                'state': 'Massachusetts',
                'creation_date': '1946-08-23'
            },
            'DRG': {
                'id': 3,
                'name': 'Miami Dragons',
                'city': 'Miami',
                'state': 'Florida',
                'creation_date': '1988-11-05'
            },
            'STR': {
                'id': 4,
                'name': 'Los Angeles Stars',
                'city': 'Los Angeles',
                'state': 'California',
                'creation_date': '1960-09-30'
            },
            'WVS': {
                'id': 5,
                'name': 'Seattle Waves',
                'city': 'Seattle',
                'state': 'Washington',
                'creation_date': '1967-12-18'
            },
            'PHX': {
                'id': 6,
                'name': 'Phoenix Flames',
                'city': 'Phoenix',
                'state': 'Arizona',
                'creation_date': '1968-03-22'
            },
            'MST': {
                'id': 7,
                'name': 'Denver Mustangs',
                'city': 'Denver',
                'state': 'Colorado',
                'creation_date': '1976-06-08'
            },
            'UNI': {
                'id': 8,
                'name': 'Brooklyn Unity',
                'city': 'Brooklyn',
                'state': 'New York',
                'creation_date': '1984-10-12'
            }
        }
        
        # Generate players and their attributes
        self._initialize_players()
        self._assign_teams_by_season()
        print("Initialization complete!")

    def _initialize_players(self):
        """Generate exactly 390 players and assign their attributes"""
        print("Generating players...")
        self.players = []
        unique_names = set()
        
        while len(unique_names) < self.players_per_season:
            name = f"{self.faker.first_name()} {self.faker.last_name()}"
            if name not in unique_names:
                unique_names.add(name)
                self.players.append(name)
        
        # Calculate number of elite players
        num_elite = int(self.players_per_season * self.elite_percentage)
        
        print(f"Assigning player archetypes ({num_elite} players per elite category)...")
        # Randomly select elite players
        all_players = set(self.players)
        self.elite_scorers = set(random.sample(list(all_players), num_elite))
        remaining_players = all_players - self.elite_scorers
        
        self.elite_defenders = set(random.sample(list(remaining_players), num_elite))
        remaining_players = remaining_players - self.elite_defenders
        
        self.elite_playmakers = set(random.sample(list(remaining_players), num_elite))
        self.regular_players = remaining_players - self.elite_playmakers
        
        # Create player dictionary with attributes
        self.player_data = {}
        for idx, player in enumerate(self.players):
            archetype = 'regular'
            if player in self.elite_scorers:
                archetype = 'elite_scorer'
            elif player in self.elite_defenders:
                archetype = 'elite_defender'
            elif player in self.elite_playmakers:
                archetype = 'elite_playmaker'
            
            self.player_data[player] = {
                'player_id': idx,
                'archetype': archetype,
                'movement_prone': random.random() < self.movement_probability
            }

    def _assign_teams_by_season(self):
        """Assign teams to players for each season"""
        print("Assigning teams for each season...")
        self.player_teams = {player: {} for player in self.players}
        
        # Initial team assignment
        for player in self.players:
            self.player_teams[player][self.seasons[0]] = random.choice(list(self.teams.keys()))
        
        # Assign teams for subsequent seasons
        for i in range(1, len(self.seasons)):
            current_season = self.seasons[i]
            previous_season = self.seasons[i-1]
            
            for player in self.players:
                previous_team = self.player_teams[player][previous_season]
                
                if self.player_data[player]['movement_prone'] and random.random() < 0.3:
                    # 30% chance to change teams for movement-prone players
                    new_team = random.choice([t for t in self.teams.keys() if t != previous_team])
                    self.player_teams[player][current_season] = new_team
                else:
                    # Stay with the same team
                    self.player_teams[player][current_season] = previous_team

    def get_team_details(self) -> List[Dict[str, Any]]:
        """Method to get team information"""
        print("\nLoading team data...")
        time.sleep(2)  # Simulate delay
        
        team_list = [
            {
                'id': details['id'],
                'name': details['name'],
                'city': details['city'],
                'state': details['state'],
                'creation_date': details['creation_date']
            }
            for details in self.teams.values()
        ]
        
        print(f"Team data loaded successfully! ({len(team_list)} teams)")
        return team_list

    def get_players(self) -> List[Dict[str, Any]]:
        """Method to get player data"""
        print("\nLoading player data...")
        time.sleep(5)  # Simulate delay
        
        players = [
            {
                'player_id': data['player_id'],
                'player_name': player,
                'archetype': data['archetype']
            }
            for player, data in self.player_data.items()
        ]
        
        print(f"Player data loaded successfully! ({len(players)} players)")
        return players

    def get_stats(self) -> List[Dict[str, Any]]:
        """Method to get player statistics"""
        stats_data = []
        
        print("\nBeginning statistics data load...")
        for season in self.seasons:
            print(f"\nLoading data for season {season}...")
            time.sleep(30)  # Simulate delay
            
            season_stats = []
            for player in self.players:
                stats = self._generate_stats(player, season)
                season_stats.append({
                    'player_id': self.player_data[player]['player_id'],
                    'season': season,
                    'team': self.player_teams[player][season],
                    **stats
                })
            
            stats_data.extend(season_stats)
            print(f"Loaded {len(season_stats)} player records for season {season}")
        
        print(f"\nAll statistics loaded successfully! ({len(stats_data)} total records)")
        return stats_data
    
    def analyze_data(players, stats):
        """Analyze the generated data"""
        print("\nData Analysis:")
        print(f"Total players: {len(players)}")
        print(f"Total stat entries: {len(stats)}")
    
    # Analyze archetypes
    archetype_counts = {}
    for player in players:
        archetype = player['archetype']
        archetype_counts[archetype] = archetype_counts.get(archetype, 0) + 1
    print("\nArchetype distribution:", archetype_counts)
    
    # Analyze team movements
    player_team_changes = {}
    for player_id in set(stat['player_id'] for stat in stats):
        player_teams = set(stat['team'] for stat in stats if stat['player_id'] == player_id)
        if len(player_teams) > 1:
            player_team_changes[player_id] = len(player_teams)
    
    print(f"\nPlayers who changed teams: {len(player_team_changes)}")
    print(f"Maximum number of teams for a single player: {max(player_team_changes.values()) if player_team_changes else 0}")

if __name__ == "__main__":
    # Test the API with timing information
    start_time = time.time()
    
    print("Starting data generation process...")
    api = BasketballAPI()
    
    print("\nFetching player data...")
    players = api.get_players()
    
    print("\nFetching statistics...")
    stats = api.get_stats()
    
    # Analyze the generated data
    analyze_data(players, stats)
    
    end_time = time.time()
    total_time = round(end_time - start_time)
    minutes = total_time // 60
    seconds = total_time % 60
    
    print(f"\nProcess completed in {minutes} minutes and {seconds} seconds")


Archetype distribution: {'regular': 333, 'elite_defender': 19, 'elite_playmaker': 19, 'elite_scorer': 19}


NameError: name 'player_id' is not defined

In [16]:
api = BasketballAPI
api


__main__.BasketballAPI

In [19]:
import random
import time
from faker import Faker
from typing import List, Dict, Any

class BasketballAPI:
    def __init__(self, players_per_season=390):
        print("Initializing Basketball API...")
        self.players_per_season = players_per_season
        self.elite_percentage = 0.05
        self.movement_probability = 0.15
        self.faker = Faker()
        Faker.seed(12345)

        self.seasons = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
        self.teams = ['SkyBirds', 'Dragons', 'Polars', 'Pandas', 'United', 'Maga', 'Moles', 'LobS']
        self.teams_data = self._generate_team_details()
        self._initialize_players()
        self._assign_teams_by_season()
        print("Initialization complete!")

    def _generate_team_details(self):
        # ... (same as before)

    def get_teams(self):
        # ... (same as before)

    def _initialize_players(self):
        # ... (same as before)

    def _assign_teams_by_season(self):
        # ... (same as before)

    def _generate_stats(self, player: str, season: str) -> Dict[str, Any]:
        archetype = self.player_data[player]['archetype']
        base_stats = {'games_played': random.randint(20, 82), 'minutes': round(random.uniform(12, 38), 1)}
        stats = {}

        if archetype == 'elite_scorer':
            stats = self._generate_scorer_stats()
        elif archetype == 'elite_defender':
            stats = self._generate_defender_stats()
        elif archetype == 'elite_playmaker':
            stats = self._generate_playmaker_stats()
        else:
            stats = self._generate_regular_stats()
        return {**base_stats, **stats}

    def _generate_scorer_stats(self):
        return {
            'points': round(random.uniform(20, 35), 1), 'rebounds': round(random.uniform(1, 8), 1),
            'assists': round(random.uniform(2, 6), 1), 'steals': round(random.uniform(0, 2), 1),
            'blocks': round(random.uniform(0, 1), 1), 'fg_pct': round(random.uniform(0.450, 0.650), 3),
            'fg3_pct': round(random.uniform(0.350, 0.450), 3), 'ft_pct': round(random.uniform(0.800, 0.950), 3)
        }

    def _generate_defender_stats(self):
        return {
            'points': round(random.uniform(8, 15), 1), 'rebounds': round(random.uniform(5, 12), 1),
            'assists': round(random.uniform(1, 4), 1), 'steals': round(random.uniform(1.5, 3), 1),
            'blocks': round(random.uniform(1, 3), 1), 'fg_pct': round(random.uniform(0.350, 0.550), 3),
            'fg3_pct': round(random.uniform(0.250, 0.380), 3), 'ft_pct': round(random.uniform(0.650, 0.850), 3)
        }

    def _generate_playmaker_stats(self):
        return {
            'points': round(random.uniform(15, 25), 1), 'rebounds': round(random.uniform(3, 7), 1),
            'assists': round(random.uniform(8, 12), 1), 'steals': round(random.uniform(1, 2.5), 1),
            'blocks': round(random.uniform(0, 1), 1), 'fg_pct': round(random.uniform(0.400, 0.550), 3),
            'fg3_pct': round(random.uniform(0.350, 0.420), 3), 'ft_pct': round(random.uniform(0.800, 0.900), 3)
        }

    def _generate_regular_stats(self):
        tier = random.choices(['rotation', 'bench', 'deep_bench'], weights=[0.5, 0.3, 0.2])[0]
        if tier == 'rotation':
            return {
                'points': round(random.uniform(8, 18), 1), 'rebounds': round(random.uniform(3, 8), 1),
                'assists': round(random.uniform(2, 5), 1), 'steals': round(random.uniform(0.5, 1.8), 1),
                'blocks': round(random.uniform(0.3, 1.2), 1), 'fg_pct': round(random.uniform(0.400, 0.520), 3),
                'fg3_pct': round(random.uniform(0.300, 0.380), 3), 'ft_pct': round(random.uniform(0.700, 0.850), 3)
            }
        elif tier == 'bench':
             return {
                'points': round(random.uniform(5, 12), 1), 'rebounds': round(random.uniform(2, 5), 1),
                'assists': round(random.uniform(1, 3), 1), 'steals': round(random.uniform(0.3, 1.2), 1),
                'blocks': round(random.uniform(0.2, 0.8), 1), 'fg_pct': round(random.uniform(0.380, 0.480), 3),
                'fg3_pct': round(random.uniform(0.280, 0.360), 3), 'ft_pct': round(random.uniform(0.650, 0.800), 3)
            }
        else:  # deep_bench
            return {
                'points': round(random.uniform(2, 8), 1), 'rebounds': round(random.uniform(1, 4), 1),
                'assists': round(random.uniform(0.5, 2), 1), 'steals': round(random.uniform(0.2, 0.8), 1),
                'blocks': round(random.uniform(0.1, 0.5), 1), 'fg_pct': round(random.uniform(0.350, 0.450), 3),
                'fg3_pct': round(random.uniform(0.250, 0.330), 3), 'ft_pct': round(random.uniform(0.600, 0.750), 3)
            }

    def get_players(self) -> List[Dict[str, Any]]:
        """Method to get player data"""
        print("\nLoading player data...")
        time.sleep(5)  # Simulate 5-second delay for player data
        
        players = [
            {
                'player_id': data['player_id'],
                'player_name': player,
                'archetype': data['archetype']
            }
            for player, data in self.player_data.items()
        ]
        
        print(f"Player data loaded successfully! ({len(players)} players)")
        return players

    def get_stats(self) -> List[Dict[str, Any]]:
        """Method to get player statistics"""
        stats_data = []
        
        print("\nBeginning statistics data load...")
        for season in self.seasons:
            print(f"\nLoading data for season {season}...")
            time.sleep(30)  # Simulate 30-second delay for each season
            
            season_stats = []
            for player in self.players:
                stats = self._generate_stats(player, season)
                season_stats.append({
                    'player_id': self.player_data[player]['player_id'],
                    'season': season,
                    'team': self.player_teams[player][season],
                    **stats
                })
            
            stats_data.extend(season_stats)
            print(f"Loaded {len(season_stats)} player records for season {season}")
        
        print(f"\nAll statistics loaded successfully! ({len(stats_data)} total records)")
        return stats_data

def analyze_data(players, stats):
    """Analyze the generated data"""
    print("\nData Analysis:")
    print(f"Total players: {len(players)}")
    print(f"Total stat entries: {len(stats)}")
    
    # Analyze archetypes
    archetype_counts = {}
    for player in players:
        archetype = player['archetype']
        archetype_counts[archetype] = archetype_counts.get(archetype, 0) + 1
    print("\nArchetype distribution:", archetype_counts)
    
    # Analyze team movements
    player_team_changes = {}
    for player_id in set(stat['player_id'] for stat in stats):
        player_teams = set(stat['team'] for stat in stats if stat['player_id'] == player_id)
        if len(player_teams) > 1:
            player_team_changes[player_id] = len(player_teams)
    
    print(f"\nPlayers who changed teams: {len(player_team_changes)}")
    print(f"Maximum number of teams for a single player: {max(player_team_changes.values()) if player_team_changes else 0}")

if __name__ == "__main__":
    # Test the API with timing information
    start_time = time.time()
    
    print("Starting data generation process...")
    api = BasketballAPI()
    
    print("\nFetching player data...")
    players = api.get_players()
    
    print("\nFetching statistics...")
    stats = api.get_stats()
    
    # Analyze the generated data
    analyze_data(players, stats)
    
    end_time = time.time()
    total_time = round(end_time - start_time)
    minutes = total_time // 60
    seconds = total_time % 60
    
    print(f"\nProcess completed in {minutes} minutes and {seconds} seconds")

IndentationError: expected an indented block after function definition on line 22 (354618193.py, line 25)